# Introduction to Social Data Science
Authors:

Simon Guldager

Astrid Waltenburg

Amelia Asp

Agnete Pade


In [1]:
import pandas as pd 
import numpy as np 
from numpy import linalg as la
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

from scipy.stats import norm
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Read data 

In [2]:
dat = pd.read_csv('LISBON_DONE.csv')
dat.index.name = 'Aparment'
dat = dat.drop(columns=['Unnamed: 4'], axis=1)
dat_filled = dat.fillna(0)

In [3]:
print(f'The data contains {dat.shape[0]} rows (aparments) and {dat.shape[1]} columns (variables).')

The data contains 389 rows (aparments) and 455 columns (variables).


In [4]:
columns = dat.columns.tolist()

columns_df = pd.DataFrame(columns=columns)

# Export the DataFrame to an Excel file
excel_file_path = 'columns_output.xlsx'  # Provide the desired file path
columns_df.to_excel(excel_file_path, index=False)

print("List of columns exported to Excel:", excel_file_path)

List of columns exported to Excel: columns_output.xlsx


# Collections of variables

In order to make the analysis simpler, it may be convenient to collect variables in sets that belong together naturally. 

In [5]:
columns = pd.read_excel('columns_categories.xlsx')

In [8]:
print("Column names:", columns.columns.tolist())

Column names: ['Label', 'Category']


In [6]:
columns = pd.read_excel('columns_categories.xlsx')

# List labels #bathroom is the reference
bedroom = []
kitchen = []
bath = []
secturity = []
entertainment = []
parking = []
host = []
work = []
family = []
outdoor = []
nearby = []
temperature = []
clothes = []
wifi =[]
drop = []

# Extract labels where Category is 1 and add them to the corresponding lists
bedroom_labels = columns.loc[columns['Category'] == 1, 'Label']
bedroom.extend(bedroom_labels)

kitchen_labels = columns.loc[columns['Category'] == 2, 'Label']
kitchen.extend(kitchen_labels)

bath_labels = columns.loc[columns['Category'] == 3, 'Label']
bath.extend(bath_labels)

security_labels = columns.loc[columns['Category'] == 4, 'Label']
secturity.extend(security_labels)

entertainment_labels = columns.loc[columns['Category'] == 5, 'Label']
entertainment.extend(entertainment_labels)

parking_labels = columns.loc[columns['Category'] == 6, 'Label']
parking.extend(parking_labels)

host_labels = columns.loc[columns['Category'] == 7, 'Label']
host.extend(host_labels)

work_labels = columns.loc[columns['Category'] == 8, 'Label']
work.extend(work_labels)

family_labels = columns.loc[columns['Category'] == 9, 'Label']
family.extend(family_labels)

outdoor_labels = columns.loc[columns['Category'] == 10, 'Label']
outdoor.extend(outdoor_labels)

nearby_labels = columns.loc[columns['Category'] == 11, 'Label']
nearby.extend(nearby_labels)

temperature_labels = columns.loc[columns['Category'] == 12, 'Label']
temperature.extend(temperature_labels)

clothes_labels = columns.loc[columns['Category'] == 13, 'Label']
clothes.extend(clothes_labels)

wifi_labels = columns.loc[columns['Category'] == 14, 'Label']
wifi.extend(wifi_labels)

drop_labels = columns.loc[columns['Category'] == 15, 'Label']
drop.extend(drop_labels)

In [7]:
wifi_labels

15      hurtig wi-fi – 205 m
42         trådløst internet
101    wi-fi – 45 mbps\nbekr
143     ethernet-forbindelse
145     hurtig wi-fi – 347 m
152    wi-fi – 19 mbps\nbekr
Name: Label, dtype: object

In [10]:
# Create a dictionary to store all the lists
vv_all = {
    'bedroom': bedroom,
    'kitchen': kitchen,
    'bath': bath,
    'security': secturity,
    'entertainment': entertainment,
    'parking': parking,
    'host': host,
    'work': work,
    'family': family,
    'outdoor': outdoor,
    'nearby': nearby,
    'temperature': temperature,
    'clothes': clothes,
    'wifi': wifi
}
list_of_lists = vv_all.values()
vv_all['all'] = [v for sublist in list_of_lists for v in sublist]

In [11]:
# convenient to keep a column of ones in the dataset
dat['constant'] = np.ones((dat.shape[0],))

# Simple OLS

In [71]:
zs = []
# Iterate through each category in vv_all and concatenate its list to zs
for category_list in vv_all.values():
    zs += category_list
ds = ['TS_dummy'] #Taylor Swift dummy
xs = ds + zs

# avoiding missings, e.g. in omtaler og host
all_vars = ['price'] + xs
I = dat[all_vars].notnull().all(1)

# extract data
X = dat.loc[I, xs].values
Z = dat.loc[I, zs].values
D = dat.loc[I, ds].values
y = dat.loc[I,'price'].values.reshape((-1,1)) * 100

# check the rank condition
K = X.shape[1]
assert np.linalg.matrix_rank(X) == X.shape[1], f'X does not have full rank'

In [23]:
# OLS: No controls
cD = np.concatenate((np.ones_like(D),D),axis = 1)

betahat = np.linalg.inv(cD.T @ cD) @ cD.T @ y
SSR_none = (y - cD@betahat).T@(y - cD@betahat)

sigma2 = (np.array(SSR_none/(cD.shape[0] - cD.shape[1])))
cov = sigma2*la.inv(cD.T@cD)
se_none = np.sqrt(cov.diagonal()).reshape(-1, 1)

OLS_none = betahat[1]

OLS_CI = OLS_none + norm.ppf(0.975)*np.array([-se_none[1].item(),se_none[1].item()])

# Adding interaction terms

In [24]:
interactions = PolynomialFeatures(interaction_only=True, include_bias = False)
Z_int = interactions.fit_transform(Z)   # Creating control variables with interaction terms
int_names = interactions.get_feature_names_out(zs)

i_idx = []      # defining list to seperate index of variables and interaction terms with variance. 
                # the looping as below is bad code, as it could be written as a one-liner

for i in range(Z_int.shape[1]):
    if np.std(Z_int[:,i]) != 0:
        i_idx.append(i)

Z_int = Z_int[:,i_idx]      # dropping interaction terms with out variance
idx_ = int_names[i_idx]     # dropping variable names of interaction terms without variance
X_int = np.concatenate((D,Z_int),axis = 1)          # Creating all variables with interaction terms

In [25]:
# y is not reshaped, as otherwise all LASSO predicts should be reshaped to ensure the correct dimension of the residuals (otherwise they will be (no aparmtnets,no apartments))
# not reshaping y has no influence on the below OLS calculations. D is flattened for the same reason
y = dat.loc[I, 'price'] * 100.0        
X = dat.loc[I, xs].values
D = D.flatten()
len_some = len(X)
contr_some = len(xs) - 1

betahat = np.linalg.inv(X.T @ X) @ X.T @ y

SSR_some = (y - X@betahat).T@(y - X@betahat)

sigma2 = (np.array(SSR_some/(X.shape[0] - X.shape[1])))
cov = sigma2*la.inv(X.T@X)
se_some = np.sqrt(cov.diagonal()).reshape(-1, 1)

OLS_some = betahat[1]
OLS_CI = OLS_some + norm.ppf(0.975)*np.array([-se_some[1].item(),se_some[1].item()])

# Data setup

In [26]:
def standardize(X):
    X_mean = np.mean(X,axis=0)
    X_std = np.std(X,axis=0)
    X_stan = (X-X_mean)/X_std
    return X_stan

X_stan = standardize(X)
Z_stan = standardize(Z)
Z_int_stan = standardize(Z_int)
X_int_stan = standardize(X_int)
D_stan = standardize(D)

# Defining penalty term

In [27]:
def BRT(X_tilde,y):
    (N,p) = X_tilde.shape
    sigma = np.std(y)
    c = 1.1
    alpha = 0.05

    penalty_BRT = (sigma * c)/np.sqrt(N)*norm.ppf(1-alpha/(2*p))

    return penalty_BRT

# Post Double Lasso

In [28]:
# Calculating penalty terms
penalty_BRTdz = BRT(Z_stan,D)
penalty_BRTdz_1 = BRT(Z_int_stan,D)

# Lasso on Taylor Swift dummy
fit_BRTdz = Lasso(alpha=penalty_BRTdz).fit(Z_stan,D) 
fit_BRTdz_1 = Lasso(alpha=penalty_BRTdz_1).fit(Z_int_stan,D) 

# Save residuals
resdz = D - fit_BRTdz.predict(Z_stan)
resdz_1 = D - fit_BRTdz_1.predict(Z_int_stan)

In [29]:
# Calculating penalty terms
penalty_BRTyx = BRT(X_stan,y)
penalty_BRTyx_1 = BRT(X_int_stan,y)

# Lasso on aparment price
fit_BRTyx = Lasso(alpha=penalty_BRTyx).fit(X_stan,y) 
fit_BRTyx_1 = Lasso(alpha=penalty_BRTyx_1).fit(X_int_stan,y) 

# Lasso coefficients
coefs = fit_BRTyx.coef_
coefs_1 = fit_BRTyx_1.coef_

# save residuals
resyxz = y-fit_BRTyx.predict(X_stan) + D_stan*coefs[0]
resyxz_1 = y-fit_BRTyx_1.predict(X_int_stan) + D_stan*coefs_1[0]

In [30]:
def PDL_ols(resdz,resyxz,d):
    denom = np.sum(resdz*d)
    num = np.sum(resdz*resyxz)
    return num/denom

def PDL_CI(resdz,resyzz):
    # Variance

    N = resyzz.shape[0]
    num = np.sum(resdz**2*resyzz**2)/N
    denom = (np.sum(resdz**2)/N)**2
    sigma2_PDL = num/denom

    # Confidence interval
    q=norm.ppf(1-0.025)
    se_PDL = np.sqrt(sigma2_PDL/N)      # calculating standard error as the squareroot of the mean variance
    CI_PDL=(((PDL-q*se_PDL).round(2),(PDL+q*se_PDL).round(2)))

    return se_PDL, CI_PDL

# Save residuals
resyzz = y - fit_BRTyx.predict(X_stan)
resyzz_1 = y - fit_BRTyx_1.predict(X_int_stan)

# Estimating Post Double Lasso
PDL = PDL_ols(resdz,resyxz,D)
PDL_1 = PDL_ols(resdz_1,resyxz_1,D)

# estimating standard errors and confidence interval for Post Double Lasso
se_PDL, CI_PDL = PDL_CI(resdz, resyzz)
se_PDL_1, CI_PDL_1 = PDL_CI(resdz_1, resyzz_1)

# Results

In [31]:
# Estimates

estimates = np.array([OLS_none[0], OLS_some, PDL, PDL_1]).round(4)
label_over_column = ['(1)','(2)','(3)','(4)']
label_column = np.array(['OLS', 'OLS', 'PDL', 'PDL'])
label_row = ['' ,'Initial $\log{(gdp)}$', 'se', 'No controls','No obs','$\lambda^{dz}$','$\lambda^{yx}$','t-statistic']
se = np.array([se_none[1].item(),se_some[1].item(),se_PDL, se_PDL_1]).round(4)
no_controls = np.array([0, contr_some, len(zs), Z_int_stan.shape[1], len(zs), Z_int_stan.shape[1]])
no_obs = np.array([len(y), len(y), len(y), len(y), len(y), len(y)])

pens_dz = np.array(['','', penalty_BRTdz.round(4), penalty_BRTdz_1.round(4)])
pens_yx = np.array(['','', penalty_BRTyx.round(4), penalty_BRTyx_1.round(4)])

t_statistic = (estimates/se).round(4)    

data = np.row_stack((label_column ,estimates, se, no_controls,no_obs, pens_dz, pens_yx,t_statistic))

df = pd.DataFrame(data = data, index = label_row, columns = label_over_column)

print(df.to_latex(escape = False))

\begin{tabular}{lllllll}
\toprule
{} &      (1) &      (2) &      (3) &      (4) &      (5) &      (6) \\
\midrule
                      &      OLS &      OLS &      PDL &      PDL &      PDL &      PDL \\
Initial $\log{(gdp)}$ &  -0.1214 &  -0.1548 &  -0.2359 &  -0.1756 &  -0.1268 &  -0.1214 \\
se                    &   0.1237 &   0.7959 &   0.2226 &   0.1932 &   0.1469 &   0.1433 \\
No controls           &        0 &       34 &       34 &      588 &       34 &      588 \\
No obs                &       70 &       70 &       70 &       70 &       70 &       70 \\
$\lambda^{dz}$        &          &          &   0.5724 &   0.7073 &   0.9139 &   1.2401 \\
$\lambda^{yx}$        &          &          &   0.5896 &   0.7267 &   1.2538 &   2.7735 \\
t-statistic           &  -0.9814 &  -0.1945 &  -1.0597 &  -0.9089 &  -0.8632 &  -0.8472 \\
\bottomrule
\end{tabular}



/var/folders/fj/trr23jhx25j8_78nf00613400000gn/T/ipykernel_6304/1949971474.py:20: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



## Table with estimates of control variables

In [34]:
# LASSO estimates - BRT with out interactions
df_tab = pd.DataFrame(data = np.row_stack((fit_BRTdz.coef_,fit_BRTyx.coef_[1:])).T, index = zs,columns = ['dz','yx']) # collecting all estimates for both stages, exluding log(gdp)
print(df_tab.loc[~(df_tab==0).all(axis=1)].round(4).to_latex(escape=False)) # removing all estimates equal to zero

\begin{tabular}{lrr}
\toprule
{} &      dz &      yx \\
\midrule
abslat      &  0.2440 &  0.0000 \\
asia        & -0.0000 &  0.0871 \\
currentinst & -0.0000 & -0.0008 \\
lh_bl       &  0.0285 &  0.0000 \\
lp_bl       &  0.0928 &  0.0000 \\
ls_bl       &  0.1319 &  0.0000 \\
\bottomrule
\end{tabular}



/var/folders/fj/trr23jhx25j8_78nf00613400000gn/T/ipykernel_6304/3417109994.py:3: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [35]:
# LASSO estimates - BRT with interaction terms
df_tab_int = pd.DataFrame(data = np.row_stack((fit_BRTdz_1.coef_,fit_BRTyx_1.coef_[1:])).T, index = idx_,columns = ['dz','yx']) # using idx_, as this variable is control variable names, adjusted for those delted due to no variance
print(df_tab_int.loc[~(df_tab_int==0).all(axis=1)].round(4).to_latex(escape=False))

\begin{tabular}{lrr}
\toprule
{} &      dz &      yx \\
\midrule
abslat marketref &  0.1245 &  0.0000 \\
abslat lp_bl     &  0.0966 &  0.0000 \\
abslat ls_bl     &  0.0144 &  0.0000 \\
cenlong asia     & -0.0000 &  0.0951 \\
lh_bl lp_bl      &  0.0629 &  0.0000 \\
lp_bl ls_bl      &  0.0818 &  0.0000 \\
\bottomrule
\end{tabular}



/var/folders/fj/trr23jhx25j8_78nf00613400000gn/T/ipykernel_6304/2173753093.py:3: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [36]:
# LASSO estimates - BCCH without interaction terms
df_tab_int = pd.DataFrame(data = np.row_stack((fit_BCCHdz.coef_,fit_BCCHyx.coef_[1:])).T, index = zs,columns = ['dz','yx']) # using idx_, as this variable is control variable names, adjusted for those delted due to no variance
print(df_tab_int.loc[~(df_tab_int==0).all(axis=1)].round(4).to_latex(escape=False))

\begin{tabular}{lrr}
\toprule
{} &      dz &   yx \\
\midrule
abslat &  0.0249 &  0.0 \\
\bottomrule
\end{tabular}



/var/folders/fj/trr23jhx25j8_78nf00613400000gn/T/ipykernel_6304/2027556558.py:3: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [37]:
# LASSO estimates - BCCH with interaction terms
df_tab_int = pd.DataFrame(data = np.row_stack((fit_BCCHdz_1.coef_,fit_BCCHyx_1.coef_[1:])).T, index = idx_,columns = ['dz','yx'])
# no non-zero control variables
print(df_tab_int.loc[~(df_tab_int==0).all(axis=1)].round(4).to_latex(escape=False))

\begin{tabular}{lrr}
\toprule
Empty DataFrame
Columns: Index(['dz', 'yx'], dtype='object')
Index: Index([], dtype='object') \\
\bottomrule
\end{tabular}



/var/folders/fj/trr23jhx25j8_78nf00613400000gn/T/ipykernel_6304/1174392149.py:4: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.

